In [1]:
import numpy as np
from siphon.catalog import TDSCatalog
import xarray as xr
# from xarray.backends import NetCDF4DataStore
# import netCDF4 as nc
import matplotlib.pylab as plt
import pandas as pd

from dask.diagnostics import ProgressBar
import dask
# dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.distributed import Client, LocalCluster




In [6]:
cat = TDSCatalog('https://www.ncei.noaa.gov/thredds/catalog/OisstBase/NetCDF/V2.0/AVHRR/catalog.html')
cat_names=cat.catalog_refs



/homes/metogra/iufarias/miniconda3/envs/ROMS/lib/python3.7/site-packages/siphon/catalog.py:293: UserWarning: URL https://www.ncei.noaa.gov/thredds/catalog/OisstBase/NetCDF/V2.0/AVHRR/catalog.html returned HTML. Changing to: https://www.ncei.noaa.gov/thredds/catalog/OisstBase/NetCDF/V2.0/AVHRR/catalog.xml
  new_url))


In [7]:
SST_year=xr.DataArray(coords=(range(len(cat_names)),range(28),range(22)),dims=['time','lat','lon'])
sst_time_string=[]

In [21]:
%%time
# ixx=10
# for ixx in range(6):
for ixx in range(388,len(cat_names)):

    new_cat_string='https://www.ncei.noaa.gov/thredds/catalog/OisstBase/NetCDF/V2.0/AVHRR/'+ str(cat_names[ixx])+'/catalog.xml'
    # new_cat_string='https://www.ncei.noaa.gov/thredds/catalog/OisstBase/NetCDF/V2.0/AVHRR/'+ str(cat_names[ixx])+'/catalog.html'
    new_cat = TDSCatalog(new_cat_string)

    ult_list=['https://www.ncei.noaa.gov/thredds/dodsC/OisstBase/NetCDF/V2.0/AVHRR/'+ str(cat_names[ixx])+'/'+ names for names in new_cat.datasets]
    ult_list.sort()

    var=xr.open_mfdataset(ult_list)#['sea_surface_temperature']
    var_sec=var.sel(lat=slice(-34,-27),lon=slice(-53 + 360,-47.5+360))
    sst_month=var_sec['sst'].groupby(var_sec.time.dt.month).mean()
    
    SST_year[ixx,:,:]=sst_month[0,0,:,:].data
    sst_time_string.append(cat_names[ixx])



syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <!DOCTYPE^ HTML PUBLIC "-//IETF//DTD HTML 2.0//EN"><html><head><title>503 Service Unavailable</title></head><body><h1>Service Unavailable</h1><p>The server is temporarily unable to service yourrequest due to maintenance downtime or capacityproblems. Please try again later.</p><p>Additionally, a 503 Service Unavailableerror was encountered while trying to use an ErrorDocument to handle the request.</p></body></html>
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <!DOCTYPE^ HTML PUBLIC "-//IETF//DTD HTML 2.0//EN"><html><head><title>503 Service Unavailable</title></head><body><h1>Service Unavailable</h1><p>The server is temporarily unable to service yourrequest due to maintenance downtime or capacityproblems. Please try again later.</p><p>Additionally, a 503 Service Unavailableerror was encountered while trying to use an ErrorDocument to handle the

CPU times: user 2min 51s, sys: 1min 3s, total: 3min 54s
Wall time: 1h 39min 49s


In [113]:
lon_er=sst_month.lon.data
lon_er-=360
lat_er=sst_month.lat.data

datestring=pd.DatetimeIndex(
    [np.datetime64(str(nombre)[0:4] + '-' + str(nombre)[4:]) for nombre in cat_names]
)


In [122]:
SST_year=SST_year.assign_coords(month_string=('time',datestring))
# SST_year.assign_coords(month_string=('time',np.arange(len(cat_names))))


SST_year['lon']=lon_er
SST_year['lat']=lat_er

In [39]:
SST_year.to_netcdf('/homes/metogra/iufarias/Alkenone_SST/data_processed_OISST/SST_year_total.nc')

# Setting up locations

In [68]:
#Locations
stations=np.array([556,557,560,561,562,564,565,566,568,569,570,571,573,574,576,579,580,581,582,583,584])
lon_s=-np.array([52.31,52.21,51.41,51.30,50.85,51.95,50.92,50.62,50.45,
              50.56,50.37,50.30,50.11,49.97,49.15,48.83,48.22,48.28,48.57,48.17,47.85])
lat_s=-np.array([33.75,33.79,33.23,33.27,33.45,32.19,32.83,33.00,33.10,
               32.29,32.48,32.53,30.99,31.10,29.83,28.63,29.29,29.22,28.03,28.15,28.24])

In [69]:
SST_stationDF=[]
for idx in range(len(stations)):
    indx_lon=np.argwhere(SST_year.lon.data<=lon_s[idx]).T[0][-1]
    indx_lat=np.argwhere(SST_year.lat.data<=lat_s[idx]).T[0][-1]

    # sst_station_sample=sst_new[:,indx_lat,indx_lon]
    SST_stationDF.append(SST_year[:,indx_lat,indx_lon].data)

In [70]:
SST_stationDF_OISST=pd.DataFrame(SST_stationDF,index=stations,columns=datestring)

# Monthly Average

In [190]:
month_string=['01_(JAN)','02_(FEV)','03_(MAR)','04_(ABR)','05_(MAI)','06_(JUN)',
              '07_(JUL)','08_(AGO)','09_(SET)','10_(OUT)','11_(NOV)','12_(DEZ)']

In [74]:
SST_stationDF_OISST.to_csv('/homes/metogra/iufarias/Alkenone_SST/data_processed_OISST/Total_OISST_climatology.csv')

In [191]:
for moi in np.arange(1,13):

    indx_month=SST_stationDF_OISST.columns.month==moi
    Eachmonth_SST_stations=SST_stationDF_OISST.iloc[:,indx_month]
    Eachmonth_SST_stations.to_csv('/homes/metogra/iufarias/Alkenone_SST/data_processed_OISST/{0}_OISST_climatology.csv'.format(month_string[moi-1]))

# Seasonal Average

In [193]:
season_index=[[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
season_string=['JFM','MAJ','JAS','OND']

In [195]:
for soi in range(4):

    indx_season=(SST_stationDF_OISST.columns.month>=season_index[soi][0])&(SST_stationDF_OISST.columns.month<=season_index[soi][2])
    Eachseason_SST_stations=SST_stationDF_OISST.iloc[:,indx_season]
    SEASON_X_SST=Eachseason_SST_stations.groupby(by=Eachseason_SST_stations.columns.year,axis=1).mean()
    SEASON_X_SST.to_csv('/homes/metogra/iufarias/Alkenone_SST/data_processed_OISST/{0}_OISST_climatology.csv'.format(season_string[soi]))